In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

def setup_driver():
    chrome_options = Options()
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

def login_and_search(username, password, query):
    driver = setup_driver()
    
    # 로그인 페이지로 이동
    login_url = "https://www.cochranelibrary.com/home?p_p_id=58&p_p_lifecycle=0&p_p_state=maximized&p_p_mode=view&saveLastPath=false&_58_struts_action=%2Flogin%2Flogin"
    driver.get(login_url)

    # 로그인 정보 입력
    username_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "_58_login"))
    )
    password_field = driver.find_element(By.ID, "_58_password")
    username_field.send_keys(username)
    password_field.send_keys(password)

    # 'Remember Me' 체크박스 해제
    remember_me_checkbox = driver.find_element(By.ID, "_58_rememberMeCheckbox")
    if remember_me_checkbox.is_selected():
        remember_me_checkbox.click()

    # 'Sign in' 버튼 클릭
    sign_in_button = driver.find_element(By.CSS_SELECTOR, "div.pull-left > button.btn.primary")
    driver.execute_script("arguments[0].click();", sign_in_button)

    # 로그인 성공 여부 확인
    try:
        WebDriverWait(driver, 10).until(EC.url_contains("home"))
        print("로그인 성공")
    except:
        print("로그인 실패, 다시 시도해보세요.")
        driver.quit()
        return

    # 로그인 후 페이지 로드 대기
    time.sleep(5)

    # 검색 페이지로 이동
    search_page_url = "https://www.cochranelibrary.com/advanced-search/search-manager"
    driver.get(search_page_url)

    # 검색어 입력
    search_box = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "textarea.search-manager-input"))
    )
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)

    # "Continue" 버튼 클릭 후 대기 시간 추가
    time.sleep(3)  # 3초 대기

    # 'Show Results' 버튼 클릭 시도
    show_results_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.search-manager-result"))
    )
    
    # 스크롤 후 잠시 대기
    driver.execute_script("arguments[0].scrollIntoView(true);", show_results_button)
    time.sleep(1)  # 스크롤 후 잠시 대기

    # 더블클릭 시도
    action = ActionChains(driver)
    action.double_click(show_results_button).perform()

    # 'Trials' 탭 클릭
    trials_tab = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Trials']"))
    )
    trials_tab.click()

    results = []
    page = 1

    while True:
        # 결과가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".search-results-item-body"))
        )

        # 결과 추출
        articles = driver.find_elements(By.CSS_SELECTOR, ".search-results-item-body")
        
        if not articles:
            break

        for article in articles:
            title = article.find_element(By.CSS_SELECTOR, "h3.result-title").text.strip()
            results.append({"Title": title})

        # 다음 페이지 확인
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.pagination-next-link")
            if "disabled" in next_button.get_attribute("class"):
                break
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)  # 페이지 로드 대기
        except:
            break

    driver.quit()
    return results

def save_to_csv(results, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Title']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

# 사용자 로그인 정보
username = "oilofmynose@gmail.com"
password = "123123a!"

# 쿼리 설정
query = '([mh "IgA Nephropathy"] OR "IgA nephropathy" OR "IgA glomerulonephritis" OR "Berger\'s disease") AND ([mh "Immunosuppressive Agents"] OR "immunosuppressive therapy" OR "immunotherapy" OR "immunomodulators") AND ([mh "Infant"] OR [mh "Child"] OR [mh "Adolescent"] OR "children" OR "pediatrics" OR "infant" OR "adolescent")'

# 검색 실행 및 결과 저장
# 검색 실행 및 결과 저장
results = login_and_search(username, password, query)
print(f"검색 완료. {len(results)}개의 결과를 가져왔습니다.")

# 결과를 화면에 출력
for result in results:
    print(result)


로그인 성공
검색 완료. 24개의 결과를 가져왔습니다.
{'Title': 'Combined therapy of enalapril and losartan attenuates histologic progression in immunoglobulin A nephropathy'}
{'Title': 'Long-term results of a randomized controlled trial in childhood IgA nephropathy'}
{'Title': 'Supportive Versus Immunosuppressive Therapy of Progressive IgA nephropathy (STOP) IgAN trial: rationale and study protocol'}
{'Title': 'Combination therapy with or without warfarin and dipyridamole for severe childhood IgA nephropathy: an RCT'}
{'Title': 'Efficacy of immunosuppressive therapy in IgA nephropathy presenting with isolated hematuria'}
{'Title': 'Steroid and cyclophosphamide in IgA nephropathy'}
{'Title': 'Effect of tonsillectomy plus steroid pulse therapy on clinical remission of IgA nephropathy: a controlled study'}
{'Title': 'Low-dose sirolimus combined with angiotensin-converting enzyme inhibitor and statin stabilizes renal function and reduces glomerular proliferation in poor prognosis IgA nephropathy'}
{'Title': 'On